# Ejercicio Módulo 2
**Inteligencia Artificial - CEIA - FIUBA**

**TEICH JUAN IGNACIO**

En este ejercicio deben implementar un algoritmo de búsqueda que no sea **Búsqueda Primero en Anchura (BFS)** para resolver el problema de la Torre de Hanoi. La nota máxima dependerá del algoritmo implementado:

- **Búsqueda Primero en Profundidad**: nota máxima 6.
- **Búsqueda de Costo Uniforme**: nota máxima 6.
- **Búsqueda de Profundidad Limitada con Profundidad Iterativa**: nota máxima 7.
- **Búsqueda Voraz usando la heurística dada en el aula virtual**: nota máxima 8.
- **Búsqueda Voraz usando una heurística desarrollada por vos**: nota máxima 9.
- **Búsqueda A\* usando la heurística dada en el aula virtual**: nota máxima 9.
- **Búsqueda A\* usando una heurística desarrollada por vos**: nota máxima 10.

La función debe devolver la salida correspondiente a la solución encontrada o `None si no se encontró una solución.

Además, debe calcular métricas de rendimiento que, como mínimo, incluyan:

- `solution_found`: `True` si se encontró la solución, `False` en caso contrario.
- `nodes_explored`: cantidad de nodos explorados (entero).
- `states_visited`: cantidad de estados distintos visitados (entero).
- `nodes_in_frontier`: cantidad de nodos que quedaron en la frontera al finalizar la ejecución (entero).
- `max_depth`: máxima profundidad explorada (entero).
- `cost_total`: costo total para encontrar la solución (float).

In [1]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi

**Heurística utilizada**. Esta heurísitica tiene 3 componentes que determinan el valor final de la función:
- el costo del camino hasta ese nodo (a mayor costo mayor heurística)
- la cantidad de discos grandes en el lugar correcto (a mayor cantidad de discos menor heurística)
- la cantidad de discos que tiene encima el disco más grande fuera de lugar (a mayor cantidad de discos encima mayor heurística)

De esta forma se cuantifica la distancia a la solución según que tan cerca están los discos que más movimientos previos requieren para ser llevados a su posición final.

In [26]:
def search_algorithm(number_disks=5) -> (NodeHanoi, dict):

    # Edite esta linea para que la list_disk use a number_disks y no quede clavado en 5
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    ##### EDITAR ESTA ZONA
    # Inicializamos las salidas, pero reemplazar con lo que se quiera usar.
    metrics = {
        "solution_found": False,
        "nodes_explored": None,
        "states_visited": None,
        "nodes_in_frontier": None,
        "max_depth": None,
        "cost_total": None,
        "nodes_repeated": None,
    }
    # Heuristic function 
    def heuristicFunction(node: NodeHanoi):
        value = node.path_cost
        # List to find the location of each disk
        locations = []
        stateList = node.state.get_state()
        for i in range(1,number_disks+1):
            if i in stateList[0]:
                locations.append(1)
            elif i in stateList[1]:
                locations.append(2)
            elif i in stateList[2]:
                locations.append(3)

        rodBiggestDisk = 0
        for i in range(number_disks, 0, -1):
            # Go through disks and substract from heuristic if the biggests disks are in the correct location
            criteria1 = False
            if locations[i - 1] != 3:
                criteria1 = True 
            elif criteria1:
                value -= 6 - i
                
            # Find the biggest disk out of place, and add to the heuristic all the disks in that rod
            if locations[i - 1] != 3:
                rodBiggestDisk = locations[i - 1]
                value += i
                continue
            if locations[i - 1] == rodBiggestDisk:
                value += i

        return value
    
    from queue import PriorityQueue
    frontier = PriorityQueue()
    visitedStates = [initial_state]
    root = NodeHanoi(state=initial_state)
    frontier.put((0,root))

    # Search loop
    nodesExplored = 0
    nodesRepeated = 0
    statesVisited = 1
    while not metrics["solution_found"]:
        # Expand the first node in the priority
        _, nodeToExpand = frontier.get()
        newNodes = nodeToExpand.expand(problem)
        # Consider this state visisted since it has been expanded and its children will be analyzed
        visitedStates.append(nodeToExpand.state)
        nodesExplored+= 1

        for newNode in newNodes:
            statesVisited += 1 # Since the state is checked it is considered visited
            # Enter the nodes that have a state not yet visited only
            if newNode.state not in visitedStates:
                # Test if solution
                if problem.goal_test(newNode.state):
                    solution = newNode
                    # print(f"Found solution! Path is:")
                    # for nodes in solution.path():
                    #     print(nodes.state)

                    metrics["solution_found"] = True
                    metrics["nodes_explored"] = nodesExplored
                    metrics["states_visited"] = statesVisited
                    metrics["nodes_in_frontier"] = len(frontier.queue)
                    metrics["max_depth"] = max([state.path_cost for _, state in frontier.queue])
                    metrics["cost_total"] = newNode.path_cost
                    metrics["nodes_repeated"] = nodesRepeated
                
                # If not solution: calculate heuristic function and add to frontier
                newNodePriority = heuristicFunction(newNode) 
                frontier.put((newNodePriority, newNode))
            else:
                nodesRepeated += 1
    # TODO: Completar con el algoritmo de búsqueda que desees implementar
    #####

    return solution, metrics

Se prueba la función:

In [29]:
solution, metrics = search_algorithm(number_disks=5)
print(solution, metrics)

<Node HanoiState:  |  | 5 4 3 2 1> {'solution_found': True, 'nodes_explored': 234, 'states_visited': 701, 'nodes_in_frontier': 25, 'max_depth': 31.0, 'cost_total': 31.0, 'nodes_repeated': 441}


Veamos las métricas:

In [6]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 234
states_visited: 701
nodes_in_frontier: 25
max_depth: 31.0
cost_total: 31.0
nodes_repeated: 441


Veamos el camino de estados desde el principio a la solución:

In [7]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

Y las acciones que el agente debería aplicar para llegar al objetivo:

In [8]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
